## HuggingFace

In [1]:
##
## https://towardsdatascience.com/transformers-for-multilabel-classification-71a1a0daf5e1

import torch
from torch.optim import AdamW
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,BertTokenizer,BertForSequenceClassification

from torchmetrics import F1Score

from tqdm import trange

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score,classification_report

import pandas as pd
import numpy as np

In [2]:
train_val = pd.read_csv("D:\\PaperPublish\\train.csv")
test_df = pd.read_csv("D:\\PaperPublish\\test.csv")

del train_val['sarcasm']
del test_df['sarcasm']

del train_val['sentiment']
del test_df['sentiment']

# del train_val['stance']
# del test_df['stance']

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Laptop GPU'

In [4]:
tokenizer = BertTokenizer.from_pretrained('asafaya/bert-base-arabic')
# from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
# model = AutoModelForMaskedLM.from_pretrained("asafaya/bert-base-arabic")

In [5]:
cols = train_val.columns
label_cols = list(cols[2:])
num_labels = len(label_cols)
print('Label columns: ', label_cols)

Label columns:  ['stance']


In [6]:
train_val['one_hot_labels'] = list(train_val[label_cols].values)

labels = list(train_val.one_hot_labels.values)
comments = list(train_val.text.values)

In [7]:
max_length = 100
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # tokenizer
# encodings = tokenizer.batch_encode_plus(comments,max_length=max_length,pad_to_max_length=True) # tokenizer's encoding method
encodings = tokenizer.batch_encode_plus(comments,max_length=max_length,pad_to_max_length=True) # tokenizer's encoding method
print('tokenizer outputs: ', encodings.keys())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


tokenizer outputs:  dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [8]:
input_ids = encodings['input_ids'] # tokenized and encoded sentences
token_type_ids = encodings['token_type_ids'] # token type ids
attention_masks = encodings['attention_mask'] # attention masks

In [9]:
# Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
label_counts = train_val.one_hot_labels.astype(str).value_counts()
one_freq = label_counts[label_counts==1].keys()
one_freq_idxs = sorted(list(train_val[train_val.one_hot_labels.astype(str).isin(one_freq)].index), reverse=True)
print('train_val label indices with only one instance: ', one_freq_idxs)

# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
one_freq_token_types = [token_type_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

train_val label indices with only one instance:  []


In [10]:
train_inputs, validation_inputs, train_labels, validation_labels, train_token_types, validation_token_types, train_masks, validation_masks = train_test_split(input_ids, labels, token_type_ids,attention_masks,
                                                            random_state=2020, test_size=0.10, stratify = labels)


In [11]:
# Add one frequency data to train data
train_inputs.extend(one_freq_input_ids)
train_labels.extend(one_freq_labels)
train_masks.extend(one_freq_attention_masks)
train_token_types.extend(one_freq_token_types)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_token_types = torch.tensor(validation_token_types)

C:\Users\Crosshair  15\AppData\Local\Temp\ipykernel_15712\203980727.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:233.)
  train_labels = torch.tensor(train_labels)


In [12]:
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [13]:
torch.save(validation_dataloader,'validation_data_loader')
torch.save(train_dataloader,'train_data_loader')


# model = BertForSequenceClassification.from_pretrained('CAMeL-Lab/bert-base-arabic-camelbert-da', num_labels=num_labels)

model = BertForSequenceClassification.from_pretrained('asafaya/bert-base-arabic', num_labels=num_labels)
model.cuda()

Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at asafaya/bert-base-ar

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [14]:
# setting custom optimization parameters. You may implement a scheduler here as well.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5)

# class_weights=class_weight.compute_class_weight('balanced',np.unique(y),y.numpy())
# class_weights=torch.tensor(class_weights,dtype=torch.float)
 
# print(class_weights)

In [15]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 25

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

    # Training

    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0 #running loss
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels, b_token_types = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()

        # # Forward pass for multiclass classification
        # outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # loss = outputs[0]
        # logits = outputs[1]

        # Forward pass for multilabel classification
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]
        
        #print everything
        
        #we might insert custom loss here
        #F1Score
        
        loss_func = BCEWithLogitsLoss()
        loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
        
        # loss_func = BCELoss() 
        # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
        train_loss_set.append(loss.item())    
        
        #print("loss1:",loss)
        #print("loss.item():",loss.item())
        #print("F1Score:")
        #F1Score()
        
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
        # scheduler.step()
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))

    ###############################################################################

    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Variables to gather full output
    logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

    # Predict
    for i, batch in enumerate(validation_dataloader):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels, b_token_types = batch
        with torch.no_grad():
            # Forward pass
            outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            b_logit_pred = outs[0]
            pred_label = torch.sigmoid(b_logit_pred)

            b_logit_pred = b_logit_pred.detach().cpu().numpy()
            pred_label = pred_label.to('cpu').numpy()
            b_labels = b_labels.to('cpu').numpy()

        tokenized_texts.append(b_input_ids)
        logit_preds.append(b_logit_pred)
        true_labels.append(b_labels)
        pred_labels.append(pred_label)

    # Flatten outputs
    pred_labels = [item for sublist in pred_labels for item in sublist]
    true_labels = [item for sublist in true_labels for item in sublist]

    # Calculate Accuracy
    threshold = 0.50
    pred_bools = [pl>threshold for pl in pred_labels]
    true_bools = [tl==1 for tl in true_labels]
    val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
    val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

    print('F1 Validation Accuracy: ', val_f1_accuracy)
    print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|                                                                                    | 0/25 [00:00<?, ?it/s]

Train loss: 0.3077105438769466


Epoch:   4%|███                                                                         | 1/25 [00:35<14:11, 35.48s/it]

F1 Validation Accuracy:  90.5982905982906
Flat Validation Accuracy:  90.5982905982906
Train loss: 0.24871229410472542


Epoch:   8%|██████                                                                      | 2/25 [01:08<13:07, 34.24s/it]

...

F1 Validation Accuracy:  90.88319088319088
Flat Validation Accuracy:  90.88319088319088
Train loss: 0.0014862629464878043


Epoch:  96%|████████████████████████████████████████████████████████████████████████   | 24/25 [13:37<00:34, 34.22s/it]

F1 Validation Accuracy:  91.16809116809117
Flat Validation Accuracy:  91.16809116809117
Train loss: 0.001565477137443271


Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 25/25 [14:11<00:00, 34.07s/it]

F1 Validation Accuracy:  90.88319088319088
Flat Validation Accuracy:  90.88319088319088


In [16]:
test_label_cols = list(test_df.columns[2:])
print('Null values: ', test_df.isnull().values.any()) #should not be any null sentences or labels
print('Same columns between train and test: ', label_cols == test_label_cols) #columns should be the same
test_df.head()

Null values:  False
Same columns between train and test:  True


,Unnamed: 0,text,stance
0,0,الوضع صار بالسعوديه يفشل لا عاد فيه شركات صاح...,1
1,1,الفساد الاداري انهك البلاد ومازلنا نعتمد علي ...,1
2,2,البي بي سي دول اوروبيه توقف تطعيم اوكسفورد اط...,1
3,3,منصه مدرستي بتسرع عمليه التحول للتعليم الالكت...,1
4,4,انا مع تمكين المراه اصلا,1


In [17]:
test_df = test_df[~test_df[test_label_cols].eq(-1).any(axis=1)] #remove irrelevant rows/comments with -1 values
test_df['one_hot_labels'] = list(test_df[test_label_cols].values)
test_df.head()

,Unnamed: 0,text,stance,one_hot_labels
0,0,الوضع صار بالسعوديه يفشل لا عاد فيه شركات صاح...,1,[1]
1,1,الفساد الاداري انهك البلاد ومازلنا نعتمد علي ...,1,[1]
2,2,البي بي سي دول اوروبيه توقف تطعيم اوكسفورد اط...,1,[1]
3,3,منصه مدرستي بتسرع عمليه التحول للتعليم الالكت...,1,[1]
4,4,انا مع تمكين المراه اصلا,1,[1]


In [18]:
# Gathering input data
test_labels = list(test_df.one_hot_labels.values)
test_comments = list(test_df.text.values)

# Encoding input data
test_encodings = tokenizer.batch_encode_plus(test_comments,max_length=max_length,pad_to_max_length=True)
test_input_ids = test_encodings['input_ids']
test_token_type_ids = test_encodings['token_type_ids']
test_attention_masks = test_encodings['attention_mask']

C:\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [19]:
# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
test_token_types = torch.tensor(test_token_type_ids)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels, test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [20]:
# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    with torch.no_grad():
        # Forward pass
        outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        b_logit_pred = outs[0]
        pred_label = torch.sigmoid(b_logit_pred)

        b_logit_pred = b_logit_pred.detach().cpu().numpy()
        pred_label = pred_label.to('cpu').numpy()
        b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

In [21]:
pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding

# Print and save classification report
print('Test F1 Score: ', f1_score(true_bools, pred_bools,average='micro'))
print('Test Precision Score: ', precision_score(true_bools, pred_bools,average='micro'))
print('Test Recall Score: ', f1_score(true_bools, pred_bools,average='micro'))



# clf_report = classification_report(true_bools,pred_bools,target_names=test_label_cols)

# print(clf_report)

Test F1 Score:  0.8982229402261712
Test Precision Score:  0.8982229402261712
Test Recall Score:  0.8982229402261712


In [22]:
# Test F1 Accuracy:  0.8975468975468975
# Test Flat Accuracy:  0.6995153473344103 

#               precision    recall  f1-score   support

#    sentiment       0.85      0.86      0.86       448
#      sarcasm       0.55      0.23      0.32        26
#       stance       0.93      0.97      0.95       560

#    micro avg       0.89      0.90      0.90      1034
#    macro avg       0.78      0.69      0.71      1034
# weighted avg       0.89      0.90      0.89      1034
#  samples avg       0.90      0.90      0.88      1034

In [23]:
## sarcasm #####
# Test F1 Score:  0.9466882067851373
# Test Precision Score:  0.9466882067851373
# Test Recall Score:  0.9466882067851373



## Sentiment ####
# Test F1 Score:  0.9239864864864864
# Test Precision Score:  0.9147157190635451
# Test Recall Score:  0.9239864864864864



## Stance ####
# Test F1 Score:  0.8982229402261712
# Test Precision Score:  0.8982229402261712
# Test Recall Score:  0.8982229402261712

In [ ]:
#tokenizer = BertTokenizer.from_pretrained('asafaya/bert-base-arabic')


In [24]:
# loss1: tensor(0.6295, device='cuda:0',grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
# loss.item(): 0.6294586658477783


In [25]:
print(train_val.sentiment.value_counts(normalize=True))
print(train_val.sarcasm.value_counts(normalize=True))
print(train_val.stance.value_counts(normalize=True))

AttributeError: 'DataFrame' object has no attribute 'sentiment'

In [ ]:
print(test_df.sentiment.value_counts(normalize=True))
print(test_df.sarcasm.value_counts(normalize=True))
print(test_df.stance.value_counts(normalize=True))

In [ ]:
#get script for enfs
#model base
#try other techniques

In [ ]:
# test_labels_np = np.array(test_labels)

# class_weights=class_weight.compute_class_weight('balanced',np.unique(test_labels),test_labels_np)
# class_weights=torch.tensor(class_weights,dtype=torch.float)
 
# print(class_weights)



In [ ]:
test_labels_np = np.array(test_labels)

In [ ]:
from sklearn.utils import class_weight

In [ ]:
np.unique(test_labels_np)

In [ ]:
test_labels

In [ ]:
###

In [ ]:
[pred,act]
[pred,act]
[pred,act]

In [ ]:
## calculating f1 score for multilabel
## multilabel f1 score
